In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from pandas_datareader import data
from scipy.stats import norm

%matplotlib inline

In [2]:
!pip install tabulate
from tabulate import tabulate

  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)


In [3]:
import csv
df = pd.read_csv('Final_stock_data_V2.csv')
df

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.1500,0.103398,1.308405e+10,...,1.25,9.550,0.1525,13.932986,22.424082,109.188247,12.97,0.171149,0.355467,0.049267
1,ABT,Abbott Laboratories,Healthcare,Increase,Above Fair Price,OverValued,Average,118.2900,0.181179,2.091710e+11,...,0.74,3.940,0.1318,22.924420,30.022842,51.234756,25.96,0.109154,0.249769,0.247649
2,ACN,Accenture plc,Technology,Increase,Above Fair Price,OverValued,Sustainable,323.9050,0.208836,2.047083e+11,...,1.21,9.613,0.1135,31.025385,33.694477,121.148794,9.45,0.166287,0.254874,0.245322
3,ADM,Archer-Daniels-Midland Company,Consumer Defensive,Increase,Above Fair Price,OverValued,Not Sustainable,76.9850,0.122815,4.327843e+10,...,0.81,4.790,0.0660,14.804809,16.072025,19.449588,36.42,0.117663,0.252289,0.150693
4,ADP,"Automatic Data Processing, Inc.",Industrials,Increase,Above Fair Price,OverValued,Sustainable,201.0100,0.182708,8.443324e+10,...,0.83,6.460,0.1371,26.483530,31.116098,90.801919,14.18,0.120094,0.272415,0.178939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,XRX,Xerox Holdings Corporation,Technology,Increase,Above Fair Price,UnderValued,No Data,20.6059,0.029031,3.678401e+09,...,1.75,1.459,-0.1600,10.302950,14.123303,0.609810,999.00,0.231929,0.405374,-0.032851
934,ZD,"Ziff Davis, Inc.",Communication Services,Increase,Above Fair Price,OverValued,No Data,102.4800,0.145864,4.941668e+09,...,1.01,4.120,0.1140,14.193907,24.873789,38.485339,999.00,0.141975,0.308007,0.055993
935,ZUMZ,Zumiez Inc.,Consumer Cyclical,Decrease,Above Fair Price,UnderValued,No Data,42.6600,0.035214,9.777245e+08,...,1.68,4.870,0.1500,9.233767,8.759754,21.330001,999.00,0.223419,0.513665,0.157523
936,ESNT,Essent Group Ltd.,Financial Services,Increase,Above Fair Price,UnderValued,No Data,44.4800,0.082041,4.912015e+09,...,1.33,5.569,0.1555,7.339934,7.987071,23.215807,999.00,0.180874,0.502764,0.045034


In [4]:
#using only undervalued stocks for our portfolio models
df2 = df.loc[df['capm_status'] == 'UnderValued']
df2

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.1500,0.103398,1.308405e+10,...,1.25,9.550,0.1525,13.932986,22.424082,109.188247,12.97,0.171149,0.355467,0.049267
5,ADSK,"Autodesk, Inc.",Technology,Increase,Below Fair Price,UnderValued,Sustainable,217.3800,0.190997,4.781773e+10,...,1.43,5.928,0.2883,31.967648,36.670040,302.038344,15.45,0.193030,0.384894,0.209161
8,AFL,AFLAC Incorporated,Financial Services,Decrease,Above Fair Price,UnderValued,Sustainable,63.0500,0.131485,4.111692e+10,...,0.93,6.390,0.0316,11.484518,9.866980,11.857047,17.30,0.132250,0.300487,0.145033
11,ALK,"Alaska Air Group, Inc.",Industrials,Increase,Above Fair Price,UnderValued,No Data,58.9750,0.141697,7.425660e+09,...,1.55,3.770,-0.2340,9.073076,15.643236,0.761055,999.00,0.207617,0.454054,-0.086043
12,ALLE,Allegion plc,Industrials,Increase,Above Fair Price,UnderValued,Average,114.1447,0.097180,1.023831e+10,...,1.14,5.086,0.0777,19.955368,22.442923,31.814578,23.59,0.157777,0.279213,0.118004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,XOMA,XOMA Corporation,Healthcare,Increase,Above Fair Price,UnderValued,No Data,20.7600,-0.034853,2.348205e+08,...,0.81,0.082,0.1900,-90.260870,253.170730,14.120154,999.00,0.117663,0.689078,0.371572
932,XP,XP Inc.,Financial Services,Increase,Above Fair Price,UnderValued,No Data,34.1100,-0.001020,1.908622e+10,...,1.91,0.986,0.0478,27.508064,34.594322,7.380346,999.00,0.251378,NaN,NaN
933,XRX,Xerox Holdings Corporation,Technology,Increase,Above Fair Price,UnderValued,No Data,20.6059,0.029031,3.678401e+09,...,1.75,1.459,-0.1600,10.302950,14.123303,0.609810,999.00,0.231929,0.405374,-0.032851
935,ZUMZ,Zumiez Inc.,Consumer Cyclical,Decrease,Above Fair Price,UnderValued,No Data,42.6600,0.035214,9.777245e+08,...,1.68,4.870,0.1500,9.233767,8.759754,21.330001,999.00,0.223419,0.513665,0.157523


In [58]:
#to meet 2/3 criteria to be selected for portfolio model
df3 = df2.loc[(df2['Earnings_expectation'] == 'Increase') & (df2['irv_status'] == 'Below Fair Price') & (df2['capm_status'] == 'UnderValued')]
df3

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
5,ADSK,"Autodesk, Inc.",Technology,Increase,Below Fair Price,UnderValued,Sustainable,217.380,0.190997,4.781773e+10,...,1.43,5.928,0.2883,31.967648,36.670040,3.020383e+02,15.45,0.193030,0.384894,0.209161
21,APTV,Aptiv PLC,Consumer Cyclical,Increase,Below Fair Price,UnderValued,Sustainable,139.980,0.197323,3.786655e+10,...,2.10,2.941,0.4626,30.364424,47.596054,6.093763e+02,13.49,0.274474,0.428095,0.175759
26,AXP,American Express Company,Financial Services,Increase,Below Fair Price,UnderValued,Not Sustainable,195.110,0.155990,1.511236e+11,...,1.24,9.573,0.4330,20.176836,20.381280,7.066180e+02,20.34,0.169933,0.343000,0.210121
32,BKR,Baker Hughes Company,Energy,Increase,Below Fair Price,UnderValued,No Data,29.185,-0.001522,2.539002e+10,...,1.62,0.180,3.4810,24.119833,162.138890,3.021507e+06,999.00,0.216126,0.473062,-0.050113
37,CAT,"Caterpillar, Inc.",Industrials,Increase,Below Fair Price,UnderValued,Not Sustainable,193.160,0.084214,1.044884e+11,...,0.90,9.348,0.3224,17.721102,20.663244,3.395254e+02,34.28,0.128603,0.314958,0.175801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,URBN,"Urban Outfitters, Inc.",Consumer Cyclical,Increase,Below Fair Price,UnderValued,No Data,25.360,-0.015427,2.482336e+09,...,1.52,2.987,3.1615,8.128205,8.490124,1.349236e+06,999.00,0.203970,0.503745,0.001591
908,USAK,"USA Truck, Inc.",Industrials,Increase,Below Fair Price,UnderValued,No Data,24.040,0.108008,2.136505e+08,...,2.09,2.057,0.2760,9.246155,11.686924,3.063938e+01,999.00,0.273259,0.638308,0.234642
910,VC,Visteon Corporation,Consumer Cyclical,Increase,Below Fair Price,UnderValued,No Data,131.390,0.155275,3.678473e+09,...,2.05,0.984,0.9518,30.204597,133.526410,7.676611e+03,999.00,0.268396,0.463500,0.066498
918,VRRM,Verra Mobility Corporation,Industrials,Increase,Below Fair Price,UnderValued,No Data,17.270,0.056159,2.695225e+09,...,1.46,0.058,0.3080,18.178947,297.758640,2.750752e+01,999.00,0.196676,NaN,NaN


In [6]:
#df for each sector

df_basic_materials = df3.loc[df['sector'] == 'Basic Materials']
df_communication = df3.loc[df['sector'] == 'Communication Services']
df_consumer_c = df3.loc[df['sector'] == 'Consumer Cyclical']
df_consumer_d = df3.loc[df['sector'] == 'Consumer Defensive']
df_energy = df3.loc[df['sector'] == 'Energy']
df_financial = df3.loc[df['sector'] == 'Financial Services']
df_healthcare = df3.loc[df['sector'] == 'Healthcare']
df_industrials = df3.loc[df['sector'] == 'Industrials']
df_real_estate = df3.loc[df['sector'] == 'Real Estate']
df_tech = df3.loc[df['sector'] == 'Technology']
df_utilites = df3.loc[df['sector'] == 'Utilities']

In [7]:
#getting the total number of stocks for each sector 

basic_total = df_basic_materials.shape[0]
comms_total = df_communication.shape[0]
consumer_c_total = df_consumer_c.shape[0]
consumer_d_total = df_consumer_d.shape[0]
energy_total = df_energy.shape[0]
financial_total = df_financial.shape[0]
healthcare_total = df_healthcare.shape[0]
industrials_total= df_industrials.shape[0]
real_estate_total = df_real_estate.shape[0]
tech_total = df_tech.shape[0]
utilities_total = df_utilites.shape[0]

print(tabulate([['Basic Materials', basic_total],['Communication Services', comms_total],['Consumer Cyclical', consumer_c_total],['Consumer Defensive', consumer_d_total], ['Energy', energy_total], ['Financial Services', financial_total], ['Healthcare', healthcare_total], ['Industrials', industrials_total], ['Real Estate', real_estate_total], ['Technology', tech_total], ['Utilities', utilities_total]],
               headers=['Sector','Total Number of Stocks']))

Sector                    Total Number of Stocks
----------------------  ------------------------
Basic Materials                               19
Communication Services                        26
Consumer Cyclical                             70
Consumer Defensive                            15
Energy                                        15
Financial Services                            65
Healthcare                                    42
Industrials                                   88
Real Estate                                    8
Technology                                    80
Utilities                                      1


In [8]:
#calculating the average historical volatility for all sectors

basic_AHV = df_basic_materials['AHV_5y'].mean()
comm_AHV = df_communication['AHV_5y'].mean()
consumer_c_AHV = df_consumer_c['AHV_5y'].mean()
consumer_d_AHV = df_consumer_d['AHV_5y'].mean()
energy_AHV = df_energy['AHV_5y'].mean()
financial_AHV = df_financial['AHV_5y'].mean()
healthcare_AHV = df_healthcare['AHV_5y'].mean()
industrials_AHV = df_industrials['AHV_5y'].mean()
real_estate_AHV = df_real_estate['AHV_5y'].mean()
tech_AHV = df_tech['AHV_5y'].mean()
utilities_AHV = df_utilites['AHV_5y'].mean()

print(tabulate([['Basic Materials', basic_AHV],['Communication Services', comm_AHV],['Consumer Cyclical', consumer_c_AHV],['Consumer Defensive', consumer_d_AHV], ['Energy', energy_AHV], ['Financial Services', financial_AHV], ['Healthcare', healthcare_AHV], ['Industrials', industrials_AHV], ['Real Estate', real_estate_AHV], ['Technology', tech_AHV], ['Utilities', utilities_AHV]],
               headers=['Sector','Historical Volatility']))

Sector                    Historical Volatility
----------------------  -----------------------
Basic Materials                        0.514
Communication Services                 0.500482
Consumer Cyclical                      0.514444
Consumer Defensive                     0.407907
Energy                                 0.523126
Financial Services                     0.408439
Healthcare                             0.552295
Industrials                            0.45136
Real Estate                            0.406957
Technology                             0.478915
Utilities                              0.260092


From above, the top 3 most volatile sectors are Healthcare (0.552295), Energy (0.523126), Consumer Cyclical (0.514444)

In [9]:
#filter dataframe to show the top 3 volatile sectors
df4 = df3.loc[(df3['sector'] == 'Healthcare') | (df3['sector'] == 'Energy') | (df3['sector'] == 'Consumer Cyclical')]
df4

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.150,0.103398,1.308405e+10,...,1.25,9.550,0.1525,13.932986,22.424082,1.091882e+02,12.97,0.171149,0.355467,0.049267
21,APTV,Aptiv PLC,Consumer Cyclical,Increase,Below Fair Price,UnderValued,Sustainable,139.980,0.197323,3.786655e+10,...,2.10,2.941,0.4626,30.364424,47.596054,6.093763e+02,13.49,0.274474,0.428095,0.175759
30,BIIB,Biogen Inc.,Healthcare,Increase,Above Fair Price,UnderValued,Not Sustainable,210.000,0.060577,3.084753e+10,...,0.44,10.217,-0.0650,11.332973,20.553978,1.630096e+01,20.62,0.072686,0.458651,-0.051561
32,BKR,Baker Hughes Company,Energy,Increase,Below Fair Price,UnderValued,No Data,29.185,-0.001522,2.539002e+10,...,1.62,0.180,3.4810,24.119833,162.138890,3.021507e+06,999.00,0.216126,0.473062,-0.050113
52,COP,ConocoPhillips,Energy,Increase,Above Fair Price,UnderValued,Not Sustainable,89.270,0.080856,1.177427e+11,...,1.62,3.687,-0.0180,11.444871,24.212095,1.077453e+01,34.87,0.216126,0.420681,0.148794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,URBN,"Urban Outfitters, Inc.",Consumer Cyclical,Increase,Below Fair Price,UnderValued,No Data,25.360,-0.015427,2.482336e+09,...,1.52,2.987,3.1615,8.128205,8.490124,1.349236e+06,999.00,0.203970,0.503745,0.001591
910,VC,Visteon Corporation,Consumer Cyclical,Increase,Below Fair Price,UnderValued,No Data,131.390,0.155275,3.678473e+09,...,2.05,0.984,0.9518,30.204597,133.526410,7.676611e+03,999.00,0.268396,0.463500,0.066498
917,VRA,"Vera Bradley, Inc.",Consumer Cyclical,Increase,Above Fair Price,UnderValued,No Data,7.790,-0.141295,2.627583e+08,...,1.82,0.607,0.1630,10.116883,12.833608,4.309673e+00,999.00,0.240438,0.593731,-0.064284
929,WW,"WW International, Inc.",Consumer Cyclical,Increase,Above Fair Price,UnderValued,No Data,10.400,-0.182074,7.282184e+08,...,1.59,0.700,-0.0670,6.011560,14.857142,7.918767e-01,999.00,0.212479,0.702709,-0.028970


In [10]:
#delete this later
df4['ticker']

0       AAP
21     APTV
30     BIIB
32      BKR
52      COP
       ... 
907    URBN
910      VC
917     VRA
929      WW
931    XOMA
Name: ticker, Length: 127, dtype: object

USE SHARPE RATIO TO GET THE VOLATILLITY ALSO CORRELATION?? lowly correlated for diversification

In [11]:
tickers = df4['ticker']

start_date = '2021-01-01'
end_date = '2021-12-31'

panel_data = data.DataReader(tickers,'yahoo', start_date, end_date)

#Use loc method to ensure we do not consider extra days of data
panel_data = panel_data.loc['2021-01-01':'2021-12-31']

closes_1y = panel_data[['Close', 'Adj Close']]

In [12]:
#correlation of the stocks

return_series_close = (closes_1y['Close'].pct_change() + 1).cumprod() -1
correlation =  return_series_close.corr()
correlation

Symbols,AAP,APTV,BIIB,BKR,COP,CTLT,CVX,DHI,F,GPC,...,TA,TACO,TILE,TNDM,TTSH,URBN,VC,VRA,WW,XOMA
Symbols,,,,,,,,,,,,,,,,,,,,,
AAP,1.000000,0.711213,-0.027165,0.302670,0.859839,0.510717,0.741432,0.753832,0.824121,0.916015,...,0.734687,-0.144828,0.791088,0.731681,0.652115,0.079466,-0.643261,0.143901,-0.483368,-0.839568
APTV,0.711213,1.000000,0.041634,0.402401,0.775247,0.553696,0.589651,0.395053,0.772878,0.583805,...,0.739784,-0.287962,0.640397,0.716013,0.592105,-0.005417,-0.242778,0.094493,-0.453483,-0.622525
BIIB,-0.027165,0.041634,1.000000,-0.125398,-0.165937,-0.094138,-0.261262,-0.052458,-0.197964,0.069155,...,-0.286053,-0.276485,0.199304,-0.279247,0.363826,0.485564,-0.114524,0.663964,0.444705,0.092838
BKR,0.302670,0.402401,-0.125398,1.000000,0.630594,0.230299,0.530959,0.022327,0.430896,0.351103,...,0.467595,0.003591,0.582845,0.331677,0.345451,-0.178990,-0.164214,-0.066484,-0.141106,-0.503995
COP,0.859839,0.775247,-0.165937,0.630594,1.000000,0.518577,0.844070,0.508260,0.869012,0.777113,...,0.830181,-0.127428,0.814029,0.758236,0.651114,-0.061135,-0.538816,0.015700,-0.509284,-0.832753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
URBN,0.079466,-0.005417,0.485564,-0.178990,-0.061135,-0.487481,0.198338,0.288034,-0.056261,0.326407,...,-0.428751,0.011360,0.274392,-0.465086,0.382782,1.000000,0.045779,0.890653,0.671880,0.192597
VC,-0.643261,-0.242778,-0.114524,-0.164214,-0.538816,-0.544723,-0.265638,-0.436214,-0.344976,-0.559183,...,-0.476109,0.285638,-0.515627,-0.499600,-0.573859,0.045779,1.000000,-0.069494,0.420887,0.727836
VRA,0.143901,0.094493,0.663964,-0.066484,0.015700,-0.330017,0.152709,0.276481,-0.025095,0.375593,...,-0.309253,-0.130262,0.400037,-0.379187,0.529116,0.890653,-0.069494,1.000000,0.607851,0.066012


In [34]:
#getting stocks with relatively low correlation --> this is for better diversification
df5 = correlation[(correlation>0.3) & (correlation<0.5)]
df5

Symbols,AAP,APTV,BIIB,BKR,COP,CTLT,CVX,DHI,F,GPC,...,TA,TACO,TILE,TNDM,TTSH,URBN,VC,VRA,WW,XOMA
Symbols,,,,,,,,,,,,,,,,,,,,,
AAP,NaN,NaN,NaN,0.302670,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APTV,NaN,NaN,NaN,0.402401,NaN,NaN,NaN,0.395053,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.363826,0.485564,NaN,NaN,0.444705,NaN
BKR,0.30267,0.402401,NaN,NaN,NaN,NaN,NaN,NaN,0.430896,0.351103,...,0.467595,NaN,NaN,0.331677,0.345451,NaN,NaN,NaN,NaN,NaN
COP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
URBN,NaN,NaN,0.485564,NaN,NaN,NaN,NaN,NaN,NaN,0.326407,...,NaN,NaN,NaN,NaN,0.382782,NaN,NaN,NaN,NaN,NaN
VC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.420887,NaN
VRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375593,...,NaN,NaN,0.400037,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
stocks = df4['ticker']

In [15]:
stocks = []

for stock in df4['ticker']:
    stocks.append(stock)

In [39]:
df5

Symbols,AAP,APTV,BIIB,BKR,COP,CTLT,CVX,DHI,F,GPC,...,TA,TACO,TILE,TNDM,TTSH,URBN,VC,VRA,WW,XOMA
Symbols,,,,,,,,,,,,,,,,,,,,,
AAP,NaN,NaN,NaN,0.302670,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APTV,NaN,NaN,NaN,0.402401,NaN,NaN,NaN,0.395053,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.363826,0.485564,NaN,NaN,0.444705,NaN
BKR,0.30267,0.402401,NaN,NaN,NaN,NaN,NaN,NaN,0.430896,0.351103,...,0.467595,NaN,NaN,0.331677,0.345451,NaN,NaN,NaN,NaN,NaN
COP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
URBN,NaN,NaN,0.485564,NaN,NaN,NaN,NaN,NaN,NaN,0.326407,...,NaN,NaN,NaN,NaN,0.382782,NaN,NaN,NaN,NaN,NaN
VC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.420887,NaN
VRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375593,...,NaN,NaN,0.400037,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
stocks = []

for i in range(127):
    

In [51]:
test = pd.DataFrame(df5.iloc[3])

,BKR
Symbols,
AAP,0.302670
APTV,0.402401
BIIB,NaN
BKR,NaN
COP,NaN
...,...
URBN,NaN
VC,NaN
VRA,NaN


In [27]:
len(stocks)

127

In [18]:
for i in range(127):
    stock = stocks[i]
    

In [19]:
df5.iloc[0]

Symbols
AAP         NaN
APTV        NaN
BIIB        NaN
BKR     0.30267
COP         NaN
         ...   
URBN        NaN
VC          NaN
VRA         NaN
WW          NaN
XOMA        NaN
Name: AAP, Length: 127, dtype: float64

In [20]:
#ignore
#get 5 year annualized returns for the stocks and pick the top 30 for the aggressive model

df4.sort_values(by=['annu_return_5y'])
df5 = df4.head(30)
df5

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.1500,0.103398,1.308405e+10,...,1.25,9.550,0.1525,13.932986,22.424082,1.091882e+02,12.97,0.171149,0.355467,0.049267
21,APTV,Aptiv PLC,Consumer Cyclical,Increase,Below Fair Price,UnderValued,Sustainable,139.9800,0.197323,3.786655e+10,...,2.10,2.941,0.4626,30.364424,47.596054,6.093763e+02,13.49,0.274474,0.428095,0.175759
30,BIIB,Biogen Inc.,Healthcare,Increase,Above Fair Price,UnderValued,Not Sustainable,210.0000,0.060577,3.084753e+10,...,0.44,10.217,-0.0650,11.332973,20.553978,1.630096e+01,20.62,0.072686,0.458651,-0.051561
32,BKR,Baker Hughes Company,Energy,Increase,Below Fair Price,UnderValued,No Data,29.1850,-0.001522,2.539002e+10,...,1.62,0.180,3.4810,24.119833,162.138890,3.021507e+06,999.00,0.216126,0.473062,-0.050113
52,COP,ConocoPhillips,Energy,Increase,Above Fair Price,UnderValued,Not Sustainable,89.2700,0.080856,1.177427e+11,...,1.62,3.687,-0.0180,11.444871,24.212095,1.077453e+01,34.87,0.216126,0.420681,0.148794
56,CTLT,"Catalent, Inc.",Healthcare,Increase,Above Fair Price,UnderValued,No Data,97.6100,0.171804,1.670966e+10,...,1.34,3.177,0.1617,23.240477,30.723953,5.346009e+01,999.00,0.182089,0.359504,0.285814
60,CVX,Chevron Corporation,Energy,Increase,Above Fair Price,UnderValued,Not Sustainable,134.3700,0.064948,2.590237e+11,...,1.27,5.173,-0.0490,14.129336,25.975256,1.215112e+01,43.22,0.173580,0.334478,0.084378
65,DHI,"D.R. Horton, Inc.",Consumer Cyclical,Increase,Above Fair Price,UnderValued,Average,81.3366,0.204622,2.897055e+10,...,1.63,11.410,0.1060,5.326562,7.128537,2.862672e+01,22.55,0.217341,0.370627,0.238886
80,F,Ford Motor Company,Consumer Cyclical,Increase,Below Fair Price,UnderValued,Average,17.4300,0.074129,6.965446e+10,...,1.08,0.713,0.7767,8.758794,24.446003,4.370325e+02,29.57,0.150484,0.375582,0.108521
91,GPC,Genuine Parts Company,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,124.8900,0.103717,1.778708e+10,...,1.08,5.609,0.0460,17.202478,22.266000,2.660739e+01,12.53,0.150484,0.294242,0.076618


In [21]:
#stocks to be used in the aggressive model
df5['ticker']

0       AAP
21     APTV
30     BIIB
32      BKR
52      COP
56     CTLT
60      CVX
65      DHI
80        F
91      GPC
93      HAS
98      HLT
106     IQV
123     LEN
130     MAR
145     MPC
159     OKE
164    PENN
173     PVH
175     PXD
182    ROST
205     UAA
208     VFC
212     WBA
217     WRK
218    XRAY
230    AGIO
242    AMWD
246    ARLP
255    ATRS
Name: ticker, dtype: object

In [22]:
tickers = ['AAP','APTV']

start_date = '2021-01-01'
end_date = '2021-12-31'

panel_data = data.DataReader(tickers,'yahoo', start_date, end_date)

Aggressive model is 70%stocks & 30% bonds or 80%stocks and 20%bonds or 100%stocks

In [23]:
#portfolio calculations, AAP 70% and APTV 50%
#portfolio returns 

weights_aap_aptv = [0.5, 0.5, 0.0]
weighted_return_series_adj_spy_tlt = weights_spy_tlt * (return_series_adj)
#Sum the weighted returns for SPY and TLT
return_series_adj_spy_tlt = weighted_return_series_adj_spy_tlt.sum(axis=1)

#Plot the weighted returns and individual returns on the same plot for comparison
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(return_series_adj_spy_tlt, label = 'weight_SPY_TLT')
# return_series_adj_spy_tlt.plot(figsize=(16,9))
ax.plot(return_series_adj['SPY'] , label = 'SPY')
ax.plot(return_series_adj['TLT'] , label = 'TLT')
ax.legend()

NameError: name 'weights_spy_tlt' is not defined

In [ ]:
pip install Nasdaq-Data-Link

In [ ]:
pip install quandl

In [ ]:
import nasdaqdatalink
import quandl

In [ ]:
quandl.get('USTREASURY/YIELD')['5 YR']
#expected return from US 5y treasury bond is 1.78